# VISUALIZATIONS FOR THE CLASSIFICATION RESULTS 

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import scipy.stats as st


In [ ]:
################# PARAMETERS ###########################

########### 01_anatomical_model_generation #############

## list of seeds
## (important for establishing a confidence interval)
seeds = [0, 1, 2, 3]  

########### 02_input_patterns_generation ###############

## Spatial correlation between MFs 
sigma = 5 

## List of MF active fractions that will be used 
f_mf = np.linspace(.05, 0.95, 10)

## Number of patterns to be simulate for each
## MF active fraction 
num_patterns = 640

########### 03_simulations ############################

## Interval used to count spikes 
## during each pattern in the EDLUT simulation
dt = interval = 0.070 

## Number of presentations/samples for each pattern
## Important parameter when noise != 0.0
n_seeds = 5 

########## 04_perceptron ##############################

## Number of classes for classification
num_classes = C = 10 

## Backpropagation parameters
gamma = 0.001 
batch_size = 32
N_epochs = 500 #Number of epochs

## Noise fraction
noise = 0.2

## Total synaptic weight that reaches each GrC from MFs
mf_grc_w = 4.00

# Inhibitory weight in GoC to GrC synapsis
goc_grc_w = 0.50 

## List of synaptic weights to simulate from MF to GoC
mf_goc_weights = [0.0, 0.10]

## List of synaptic weights to simulate from GrC to GoC
grc_goc_weights = [0.02]

########## 05_visualizations ############################
#### Combination of weights MF - GoC and GrC - GoC ######
comb_weights = [[0.0,  0.02], [0.1, 0.02]]
n = len(comb_weights)

#########################################################

## Functions to get the accuracies already calculated

In [ ]:
sys.path.insert(1, 'accuracies')

def get_acc(seed, elem, mf_goc_w, grc_goc_w, acc_type):
    
    #directories used
    RESULTS_DIR = '../results/accuracies' + '/seed' + str(seed) 
    params_file_name = RESULTS_DIR + '/params_file.csv' 
    
    #reading dataframe
    df = pd.read_csv(params_file_name)
    
    #creating dictionary
    my_list_params = {'positions_seed': seed, 
                      'perceptron_seed': seed, 
                      'sigma': sigma, 
                      'fraction': elem, 
                      'noise': noise, 
                      'mf_grc_w': mf_grc_w,
                      'mf_goc_w': mf_goc_w,
                      'grc_goc_w': grc_goc_w,
                      'goc_grc_w': goc_grc_w,
                      'interval': interval,
                      'N_epochs': N_epochs,
                      'C': C,
                      'num_patterns': num_patterns,
                      'n_seeds': n_seeds,
                      'gamma': gamma,
                      'batch_size': batch_size,
                      'type': acc_type
                     }

    ## getting rows matching the dictionary
    for param in my_list_params:
        df = df.loc[(df[param] == my_list_params[param])]
        
    result = df['filename'].tolist()

    ## getting the filename associated to the last row
    filename_result = ''
    if result == []:
        print('Not found')
    else:
        # take the last element from the list (the one in the bottom of the table)
        filename_result = result[-1]
    
    ## getting accuracy values
    acc = pd.read_csv(os.path.join(RESULTS_DIR, filename_result))

    acc = acc.drop(acc.columns[0], axis = 1)   
    acc.index = ['MF', 'GRC']

    return acc

In [ ]:

# Function to get GrC samples
def get_samples_grc(seed, elem, mf_goc_w, grc_goc_w):

    x_grc = np.loadtxt('../data/spikes/seed' + str(seed) 
            + '/sigma_' + str(sigma) 
            + '/noise_' + str(noise)
            + '/grc'
            + '/mf_grc_' + str(mf_grc_w) 
            + '/mf_goc_w_' + str(mf_goc_w)
            + '/goc_grc_w_' + str(goc_grc_w)
            + '/grc_goc_w_' + str(grc_goc_w)
            +'/f_mf_' + str(elem) 
            + '/i' + str(interval) + '_grc_s' + str(0) + '.csv')
    
    for k in range(n_seeds-1): 
        
        x_grc += np.loadtxt('../data/spikes/seed' + str(seed) 
            + '/sigma_' + str(sigma) 
            + '/noise_' + str(noise)
            + '/grc'
            + '/mf_grc_' + str(mf_grc_w) 
            + '/mf_goc_w_' + str(mf_goc_w)
            + '/goc_grc_w_' + str(goc_grc_w)
            + '/grc_goc_w_' + str(grc_goc_w)
            +'/f_mf_' + str(elem) 
            + '/i' + str(interval) + '_grc_s' + str(k+1) + '.csv')
        
    x_grc = x_grc/n_seeds

    return x_grc



# Plotting results

In [ ]:
## Colors used to plot the different weights combinations

T = n
plt.tick_params(left = False, right = False , labelleft = False , 
                labelbottom = False, bottom = False)
colors = ['darkblue','green', 'orange'] 
cm = LinearSegmentedColormap.from_list(
        "Custom", colors, N=T)
mat = np.indices((1,T))[1]
plt.imshow(mat, cmap=cm)
colors = []
for i in range(T): 

    colors.append(cm(i))


## TRAINING

In [ ]:
## Creating the directory for accuracies
PLOTS_DIR = '../results/plots'  
os.makedirs(PLOTS_DIR, exist_ok=True)

In [ ]:
## Retrieving training accuracy values 
## Calculating means and standard deviations 

matrix_grc = np.zeros((f_mf.shape[0], n, len(seeds)))
matrix_mf = np.zeros((f_mf.shape[0], 1, len(seeds)))
mean_grc = np.zeros((f_mf.shape[0], n))
mean_mf = np.zeros((f_mf.shape[0], 1))
train_mean_grc = np.zeros((f_mf.shape[0], n))
train_mean_mf = np.zeros((f_mf.shape[0], 1))
std_grc = np.zeros((f_mf.shape[0], n))
std_mf = np.zeros((f_mf.shape[0], 1))

for x in range(len(comb_weights)):
    mf_goc_w = comb_weights[x][0]
    grc_goc_w = comb_weights[x][1]

    for xs in range(len(seeds)):
        s = seeds[xs]
        
        for i in range(len(f_mf)): 

            elem = round(f_mf[i],2)
            acc = get_acc(s, elem, mf_goc_w, grc_goc_w, 'train')
            acc_grc = acc.iloc[1]
            acc_mf = acc.iloc[0]

            matrix_grc[i, x, xs] = acc_grc[int(acc_grc.last_valid_index())]
            matrix_mf[i, 0, xs] = acc_mf[int(acc_mf.last_valid_index())]

    mean_grc[:,x] = np.mean(matrix_grc[:,x,:], axis = 1)                    
    std_grc[:,x] = np.std(matrix_grc[:,x,:], axis = 1)


mean_mf[:,0] = np.mean(matrix_mf[:,0,:], axis = 1)             
std_mf[:,0] = np.std(matrix_mf[:,0,:], axis = 1)
        
    
train_mean_grc = mean_grc
train_mean_mf = mean_mf

In [ ]:
## Calculating confidence intervals 

conf_int_grc = np.zeros((matrix_grc.shape[0], matrix_grc.shape[1], 2))
conf_int_mf = np.zeros((matrix_grc.shape[0], 1, 2))

for j in range(matrix_grc.shape[1]):
    for i in range(matrix_grc.shape[0]):
        #creates 95% confidence interval
        data_grc = matrix_grc[i,j,:]
        data_mf = matrix_mf[i,0,:]
        conf_int_grc[i,j,0], conf_int_grc[i,j,1] = st.t.interval(alpha=0.95, df=len(data_grc)-1, loc=np.mean(data_grc), scale=st.sem(data_grc))
        conf_int_mf[i,0,0], conf_int_mf[i,0,1] = st.t.interval(alpha=0.95, df=len(data_mf)-1, loc=np.mean(data_mf), scale=st.sem(data_mf))

In [ ]:
## Plotting training accuracies vs MF activation percentages

fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize = (6,4), sharex = True)
plt.rcParams["savefig.facecolor"] = "white"
for c in range(len(comb_weights)):
    
    mf_goc_w = comb_weights[c][0]
    grc_goc_w = comb_weights[c][1]
        
    ax.plot(f_mf*100, mean_grc[:,c], label = 'GRC (MF-GOC: ' + str(mf_goc_w) + ') (GRC-GOC: ' + str(grc_goc_w) + ')', c=colors[c])
    ax.errorbar(f_mf*100, mean_grc[:,c], fmt='-o', c = colors[c])
    stdmin = conf_int_grc[:,c,0]
    stdmax = conf_int_grc[:,c,1]
    ax.fill_between(f_mf*100, stdmin, stdmax, alpha=0.5, edgecolor=colors[c], facecolor=colors[c])


ax.set_ylim([0.00, 1.10])

ax.set_ylabel('Accuracy', fontsize = 16)
ax.set_xlabel('MF activation (%)', fontsize = 16)
ax.set_xticks(f_mf*100)
ax.plot(f_mf*100, mean_mf, label = 'MF', c='red') 
ax.errorbar(f_mf*100, mean_mf[:,0], yerr=std_mf[:,0], fmt='-o', c = 'red')
stdmin = conf_int_mf[:,0,0]
stdmax = conf_int_mf[:,0,1]
ax.fill_between(f_mf*100, stdmin, stdmax, alpha=0.5, edgecolor='red', facecolor='red')


ax.spines[['right', 'top']].set_visible(False)
ax.tick_params('both', labelsize = 15)
ax.legend(loc=(1.04, 0), fontsize = 12)

fig.suptitle("TRAINING (Epochs =" + str(N_epochs) + ") (" + str(noise*100) + "% noise)" , fontsize=16)

In [ ]:
path = PLOTS_DIR + '/sigma_' + str(sigma) + '/noise_' + str(noise) 
os.makedirs(path, exist_ok=True)
name = '_N' + str(N_epochs) + '_C' + str(C) + '_NP' + str(num_patterns) +  '_samples' + str(n_seeds) + '_sb' + str(batch_size) + '_g' + str(gamma) + '_i' + str(interval) + '.png'

fig.savefig(path + '/acc_train_' + name,  bbox_inches = 'tight', dpi = 200)

## TEST

In [ ]:
## Retrieving test accuracy values 
## Calculating means and standard deviations 
matrix_grc = np.zeros((f_mf.shape[0], n, len(seeds)))
matrix_mf = np.zeros((f_mf.shape[0], 1, len(seeds)))
mean_grc = np.zeros((f_mf.shape[0], n))
mean_mf = np.zeros((f_mf.shape[0], 1))
std_grc = np.zeros((f_mf.shape[0], n))
std_mf = np.zeros((f_mf.shape[0], 1))
grc_firing_rate = np.zeros((f_mf.shape[0], n, len(seeds)))
mean_grc_firing_rate = np.zeros((f_mf.shape[0], n))
std_grc_firing_rate = np.zeros((f_mf.shape[0], n))

for x in range(len(comb_weights)):
    mf_goc_w = comb_weights[x][0]
    grc_goc_w = comb_weights[x][1]

    for xs in range(len(seeds)):
        s = seeds[xs]

        for i in range(len(f_mf)): 

            elem = round(f_mf[i],2)
            acc = get_acc(s, elem, mf_goc_w, grc_goc_w, 'test')
            acc_grc = acc.iloc[1]
            acc_mf = acc.iloc[0]
            x_grc = get_samples_grc(s, f_mf[i], mf_goc_w, grc_goc_w)
            grc_firing_rate[i, x, xs] = sum((sum(x_grc)/x_grc.shape[0])/interval)/num_patterns

            matrix_grc[i, x, xs] = acc_grc[int(acc_grc.last_valid_index())]
            matrix_mf[i, 0, xs] = acc_mf[int(acc_mf.last_valid_index())]

    mean_grc[:,x] = np.mean(matrix_grc[:,x,:], axis = 1)                    
    std_grc[:,x] = np.std(matrix_grc[:,x,:], axis = 1)
    mean_grc_firing_rate[:,x] = np.mean(grc_firing_rate[:,x,:], axis = 1)
    std_grc_firing_rate[:,x] = np.std(grc_firing_rate[:,x,:], axis = 1)

mean_mf[:,0] = np.mean(matrix_mf[:,0,:], axis = 1)             
std_mf[:,0] = np.std(matrix_mf[:,0,:], axis = 1)

In [ ]:
## Calculating confidence intervals
conf_int_grc = np.zeros((matrix_grc.shape[0], matrix_grc.shape[1], 2))
conf_int_mf = np.zeros((matrix_grc.shape[0], 1, 2))


for j in range(matrix_grc.shape[1]):
    for i in range(matrix_grc.shape[0]):
        #create 95% confidence interval for population mean weight
        data_grc = matrix_grc[i,j,:]
        data_mf = matrix_mf[i,0,:]
        conf_int_grc[i,j,0], conf_int_grc[i,j,1] = st.t.interval(alpha=0.95, df=len(data_grc)-1, loc=np.mean(data_grc), scale=st.sem(data_grc))
        conf_int_mf[i,0,0], conf_int_mf[i,0,1] = st.t.interval(alpha=0.95, df=len(data_mf)-1, loc=np.mean(data_mf), scale=st.sem(data_mf))

In [ ]:
## Plotting test accuracies vs MF activation percentages

fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize = (6,4), sharex = True)

for c in range(len(comb_weights)):
    
    mf_goc_w = comb_weights[c][0]
    grc_goc_w = comb_weights[c][1]
        
    ax.plot(f_mf*100, mean_grc[:,c], label = 'GRC (MF-GOC: ' + str(mf_goc_w) + ') (GRC-GOC: ' + str(grc_goc_w) + ')', c=colors[c])
    ax.plot(f_mf*100, train_mean_grc[:,c], c=colors[c], ls = '-.')
    ax.errorbar(f_mf*100, mean_grc[:,c], fmt='-o', c = colors[c])
    stdmin = conf_int_grc[:,c,0]
    stdmax = conf_int_grc[:,c,1]
    ax.fill_between(f_mf*100, stdmin, stdmax, alpha=0.5, edgecolor=colors[c], facecolor=colors[c])


ax.set_ylim([0.00, 1.10])

ax.set_ylabel('Accuracy', fontsize = 16)
ax.set_xlabel('MF activation (%)', fontsize = 16)
ax.set_xticks(f_mf*100)
ax.plot(f_mf*100, mean_mf, label = 'MF', c='red') 

ax.errorbar(f_mf*100, mean_mf[:,0], yerr=std_mf[:,0], fmt='-o', c = 'red')
stdmin = conf_int_mf[:,0,0]
stdmax = conf_int_mf[:,0,1]
ax.fill_between(f_mf*100, stdmin, stdmax, alpha=0.5, edgecolor='red', facecolor='red')


ax.spines[['right', 'top']].set_visible(False)
ax.tick_params('both', labelsize = 15)
ax.legend(loc=(1.04, 0), fontsize = 12)

fig.suptitle("TEST (Epochs =" + str(N_epochs) + ") (" + str(noise*100) + "% noise)" , fontsize=16)


In [ ]:
## Saving files

path = PLOTS_DIR + '/sigma_' + str(sigma) + '/noise_' + str(noise) 
os.makedirs(path, exist_ok=True)
name = '_N' + str(N_epochs) + '_C' + str(C) + '_NP' + str(num_patterns) +  '_samples' + str(n_seeds) + '_sb' + str(batch_size) + '_g' + str(gamma) + '_i' + str(interval) + '.png'

fig.savefig(path + '/acc_test_' + name,  bbox_inches = 'tight', dpi = 200)

## FIRING RATES PLOTS

In [ ]:
## Plotting test accuracies vs average neuron firing rate

fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize = (6,4), sharex = True)

name = '_N' + str(N_epochs) + '_c' + str(C) + '_np' + str(num_patterns) +  '_pres' + str(n_seeds) + '_sb' + str(batch_size) + '_g' + str(gamma) + '_i' + str(interval) + '.png'


sys.path.insert(1, 'plots')

c = 0

for c in range(len(comb_weights)):

        mf_goc_w = comb_weights[c][0]
        grc_goc_w = comb_weights[c][1]
        
        ax.plot(mean_grc_firing_rate[:,c], mean_grc[:,c], label = 'GRC (MF-GOC: ' + str(mf_goc_w) + ') (GRC-GOC: ' + str(round(grc_goc_w,3)) + ')', c=colors[c])
        ax.errorbar(mean_grc_firing_rate[:,c], mean_grc[:,c], xerr = std_grc_firing_rate[:,0], fmt='-o', c = colors[c])
        ax.set_ylim([0.00, 1.10])
        stdmin = conf_int_grc[:,c,0]
        stdmax = conf_int_grc[:,c,1]
        ax.fill_between(mean_grc_firing_rate[:,c], stdmin, stdmax, alpha=0.5, edgecolor=colors[c], facecolor=colors[c])

        c += 1


ax.set_ylabel('Accuracy', fontsize = 16)
ax.set_xlabel('Neuron average firing rate (Hz)', fontsize = 16)

ax.plot(f_mf*50, mean_mf, label = 'MF', c='red') 
ax.errorbar(f_mf*50, mean_mf[:,0], yerr=std_mf[:,0], fmt='-o', c = 'red')
stdmin = conf_int_mf[:,0,0]
stdmax = conf_int_mf[:,0,1]
ax.fill_between(f_mf*50, stdmin, stdmax, alpha=0.5, edgecolor='red', facecolor='red')
ax.legend(loc=(1.04, 0), fontsize = 12)
ax.spines[['right', 'top']].set_visible(False)
ax.tick_params('both', labelsize = 16)


fig.suptitle("TEST (Epochs =" + str(N_epochs) + ") (" + str(noise*100) + "% noise)" , fontsize=16)

In [ ]:
## Saving plot

path = PLOTS_DIR + '/sigma_' + str(sigma) + '/noise_' + str(noise) 
os.makedirs(path, exist_ok=True)
name = '_N' + str(N_epochs) + '_C' + str(C) + '_NP' + str(num_patterns) +  '_samples' + str(n_seeds) + '_sb' + str(batch_size) + '_g' + str(gamma) + '_i' + str(interval) + '.png'

fig.savefig(path + '/acc_test_fir_rates_' + name,  bbox_inches = 'tight', dpi = 200)